In [98]:
import csv
import pandas as pd
import numpy as np

In [1]:
escitalopram_id = "100002771" #"CID100002771"
atc1 = "N06AB04"
atc2 = "N06AB10"

In [27]:
dataset_path = "/home/samuel/NYU/BDS/project/meddra/"

def read_tsv_file(file_path, headers = True):
    try:
        data = []
        with open(file_path, 'r', newline='', encoding='utf-8') as file:
            # Using the CSV reader with the tab delimiter
            reader = csv.reader(file, delimiter='\t')

            if headers == True:
                # Reading headers
                headers = next(reader)
                print("Headers:", headers)

            # Reading data rows
            for row in reader:
                data.append(row)

        # Create a Pandas DataFrame using the headers and data
        if headers == True:
            df = pd.DataFrame(data, columns=headers)
        else:
            df = pd.DataFrame(data) #, columns=headers)
        return df

    except FileNotFoundError:
        print(f"Error: File not found at '{file_path}'")
    except Exception as e:
        print(f"An error occurred: {e}")
meddra_file_path = dataset_path + 'meddra_all_se.tsv'
meddra_df = read_tsv_file(meddra_file_path)
escitalopram_sides_table = meddra_df[meddra_df[0] == escitalopram_id]
escitalopram_sides = escitalopram_sides_table[5].unique()

Headers: ['CID100000085', 'CID000010917', 'C0000729', 'LLT', 'C0000729', 'Abdominal cramps']


In [61]:
processed_dataset = pd.read_csv("../review_dataset/drugsComTrain_processed.csv")
escitalopram_dataset = processed_dataset[processed_dataset["drugName"] == "Escitalopram"]
escitalopram_dataset

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount,processed_review
203,140891,Escitalopram,Generalized Anxiety Disorde,"""The time I been on lexapro I had mild side ef...",7.0,"April 27, 2016",13,"""the time lexapro mild side effect like yawn i..."
237,214628,Escitalopram,Generalized Anxiety Disorde,"""This drug is pretty great for depression, how...",5.0,"August 14, 2017",7,"""thi drug pretti great depression, howev serio..."
252,141047,Escitalopram,Anxiety,"""Day 2. Taking for major anxiety/ slight depre...",10.0,"October 6, 2015",11,"""day 2. take major anxiety/ slight depression...."
314,142104,Escitalopram,Depression,"""I&#039;ve been on lexapro for several years. ...",8.0,"March 9, 2009",376,"""i'v lexapro sever years. say initi side effec..."
405,141645,Escitalopram,Depression,"""I am a 35 year-old female. I&#039;ve always b...",10.0,"November 3, 2012",81,"""i 35 year-old female. i'v alway melanchol per..."
...,...,...,...,...,...,...,...,...
160596,141107,Escitalopram,Depression,"""It works really good. I also have anxiety and...",9.0,"August 10, 2015",65,"""it work realli good. also anxieti part helps...."
160625,141426,Escitalopram,Anxiety,"""I feel compelled to share my experience becau...",9.0,"January 27, 2014",41,"""i feel compel share experi know other like me..."
160687,140846,Escitalopram,Depression,"""I have taken Lexapro for 10 years. I had seve...",9.0,"April 20, 2016",32,"""i taken lexapro 10 years. sever depression/an..."
160716,141419,Escitalopram,Depression,"""Sexually frustrated me and I was a little bit...",6.0,"February 9, 2014",36,"""sexual frustrat littl bit aggress come onto i..."


In [59]:
escitalopram_sides_counted = pd.DataFrame(escitalopram_sides, columns=["side_effect_name"]).copy()
escitalopram_sides_counted["count"] = 0
for review in escitalopram_dataset["review"]:
    for side_effect_name in escitalopram_sides_counted["side_effect_name"]:
        if side_effect_name.lower() in review.lower():
            escitalopram_sides_counted.loc[escitalopram_sides_counted["side_effect_name"] == side_effect_name, "count"] += 1
escitalopram_sides_counted.sort_values(by = "count", ascending=False)

,side_effect_name,count
17,Anxiety,839
355,Depression,462
607,Panic attack,217
183,Nausea,126
120,Headache,113
...,...,...
313,Phosphatase alkaline increased,0
314,Blood alkaline phosphatase increased,0
315,Prothrombin level increased,0
316,Electrocardiogram QT prolonged,0


In [122]:
relevant_side_effect = list(escitalopram_sides_counted[escitalopram_sides_counted["count"] > 5]["side_effect_name"])
for relevant_effect in relevant_side_effect:
    escitalopram_dataset.loc[:, relevant_effect] = False
    for index,  review in escitalopram_dataset["review"].items():
        if relevant_effect.lower() in review.lower():
            escitalopram_dataset.at[index, relevant_effect] = True
escitalopram_dataset

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount,processed_review,Anxiety,Chest pain,...,Anger,Fibromyalgia,Hunger,Irritability,Lethargy,Night sweats,Panic attack,Upset stomach,Crying,Anxiety attack
203,140891,Escitalopram,Generalized Anxiety Disorde,"""The time I been on lexapro I had mild side ef...",7.0,"April 27, 2016",13,"""the time lexapro mild side effect like yawn i...",False,False,...,False,False,False,False,False,False,False,False,False,False
237,214628,Escitalopram,Generalized Anxiety Disorde,"""This drug is pretty great for depression, how...",5.0,"August 14, 2017",7,"""thi drug pretti great depression, howev serio...",True,False,...,False,False,True,False,False,False,False,False,False,False
252,141047,Escitalopram,Anxiety,"""Day 2. Taking for major anxiety/ slight depre...",10.0,"October 6, 2015",11,"""day 2. take major anxiety/ slight depression....",True,False,...,False,False,False,False,False,False,False,False,False,False
314,142104,Escitalopram,Depression,"""I&#039;ve been on lexapro for several years. ...",8.0,"March 9, 2009",376,"""i'v lexapro sever years. say initi side effec...",False,False,...,False,False,False,False,False,False,False,False,False,False
405,141645,Escitalopram,Depression,"""I am a 35 year-old female. I&#039;ve always b...",10.0,"November 3, 2012",81,"""i 35 year-old female. i'v alway melanchol per...",True,False,...,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160596,141107,Escitalopram,Depression,"""It works really good. I also have anxiety and...",9.0,"August 10, 2015",65,"""it work realli good. also anxieti part helps....",True,False,...,False,False,False,False,False,False,False,False,False,False
160625,141426,Escitalopram,Anxiety,"""I feel compelled to share my experience becau...",9.0,"January 27, 2014",41,"""i feel compel share experi know other like me...",True,False,...,False,False,False,False,False,False,False,False,False,False
160687,140846,Escitalopram,Depression,"""I have taken Lexapro for 10 years. I had seve...",9.0,"April 20, 2016",32,"""i taken lexapro 10 years. sever depression/an...",True,False,...,False,False,False,False,False,False,False,False,False,False
160716,141419,Escitalopram,Depression,"""Sexually frustrated me and I was a little bit...",6.0,"February 9, 2014",36,"""sexual frustrat littl bit aggress come onto i...",False,False,...,False,False,False,False,False,False,False,False,False,False


In [130]:
reg_df = escitalopram_dataset.drop(columns=["Unnamed: 0", "drugName", "condition", "review", "date", "usefulCount", "processed_review"])

In [131]:
reg_df = reg_df[(reg_df.drop(columns=["rating"]) == True).any(axis = 1)]
#reg_df = reg_df[(reg_df.drop(columns=["rating"]).sum(axis=1) >= 3)]

In [132]:
reg_df.shape

(1059, 39)

In [133]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

X = reg_df.drop(columns=['rating'])  # Assuming 'review' is the target column
y = reg_df['rating']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the linear regression model
model = LinearRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

# Predict the review ratings for the test data
y_pred = model.predict(X_test)

# Calculate Mean Squared Error to evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 7.135062374951185


In [134]:
np.unique(y_pred)

array([ 4.03199534,  5.03929028,  5.807346  ,  5.86164673,  5.89121096,
        6.11921214,  6.19846719,  6.20939192,  6.36564997,  6.44374153,
        6.50324716,  6.52309854,  6.61747141,  6.65415879,  6.67628199,
        6.69850765,  6.81160735,  6.91306464,  6.92290338,  7.11878248,
        7.15506722,  7.17128768,  7.17350383,  7.24773319,  7.26331945,
        7.28284975,  7.28604636,  7.34758304,  7.35953046,  7.36530141,
        7.37588485,  7.42002276,  7.44214596,  7.44349493,  7.46561813,
        7.51721797,  7.55390535,  7.69001386,  7.70954093,  7.71135392,
        7.71213706,  7.77457803,  7.80042428,  7.84128376,  7.88904609,
        7.97685386,  7.99141804,  7.99718898,  8.02748221,  8.04649466,
        8.05191033,  8.0526556 ,  8.12776549,  8.14087084,  8.1455183 ,
        8.1488666 ,  8.14988869,  8.15463755,  8.15690509,  8.16445286,
        8.18657607,  8.27670592,  8.30296687,  8.31976933,  8.32190143,
        8.34402464,  8.4051743 ,  8.4272975 ,  8.43231186,  8.45

In [76]:
relevant_side_effect = list(escitalopram_sides_counted[escitalopram_sides_counted["count"] > 5]["side_effect_name"])
feature_df = pd.DataFrame(columns = ["Score"].append(relevant_side_effect))
print(feature_df)


Empty DataFrame
Columns: []
Index: []
